# Bivariate Examples:
 * Correlation
 * Regression and fitting curves
 * Confidence intervals

Use Cat Retina data "catretina.csv" [Lia _et al._, Science (1987)](https://www.science.org/doi/10.1126/science.3576202)
* We'll consider 'cpRatio' as a function of 'retinarea' for this example

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
from scipy.optimize import curve_fit

### Get and plot the data
* several columns. We use 'retinarea' as the independent variably (x axis), and 'cpRatio' as the dependent variable (y axis)

In [ ]:
filename = "catretina.csv"
data = np.genfromtxt(filename, delimiter=",", names=True, skip_header=1)
collumns = list(data.dtype.names)

print(collumns)

xdata = data["retinarea"]
ydata = data["cpRatio"]

In [ ]:
plt.plot(xdata, ydata, "bx")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.show()

### Correlation coeficient

Simple test: are the data correlated?

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

In [ ]:
pearson = stats.pearsonr(xdata, ydata)
print(
    f"Pearson correlation coef: {pearson.statistic:.3f}, p-value: {pearson.pvalue:.2e}"
)

spearman = stats.spearmanr(xdata, ydata)
print(
    f"Pearson correlation coef: {spearman.statistic:.3f}, p-value: {spearman.pvalue:.2e}"
)

### Fit polynomial
* Fit linear, quadratic, cubic polynomials
* plot
* nb: I'm using the older library: https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html
* There's a newer one: https://numpy.org/doc/stable/reference/generated/numpy.polynomial.polynomial.Polynomial.fit.html

In [ ]:
coefs1 = np.polyfit(xdata, ydata, 1)
coefs2 = np.polyfit(xdata, ydata, 2)
coefs3 = np.polyfit(xdata, ydata, 3)

# Takes the coefs from the fit, and creates callable polynomial functions
y1 = np.poly1d(coefs1)
y2 = np.poly1d(coefs2)
y3 = np.poly1d(coefs3)

In [ ]:
# Create denser x-grid for plotting
x = np.linspace(min(xdata), max(xdata), 100)

plt.plot(data["retinarea"], data["cpRatio"], "bx")
plt.plot(x, y1(x), label="linear")
plt.plot(x, y2(x), label="quadratic")
plt.plot(x, y3(x), label="cubic")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()

### Residuals:

In [ ]:
res1 = ydata - y1(xdata)


plt.plot(xdata, res1, "bx")
plt.title("Linear-fit residuals")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio residuals (linear)")
plt.show()

In [ ]:
res2 = ydata - y2(xdata)

plt.plot(xdata, res2, "bx")
plt.title("Quadratic-fit residuals")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio residuals (quadratic)")
plt.show()

#### Define R^2

$$
  R^2 = 1 - \frac{SS_{\rm res}}{SStot}
$$

where:

$$
  SS_{\rm res} = \sum_i [y_i - y(x_i)]^2
$$

is the sum of square residuals is, and

$$
  SS_{\rm tot} = \sum_i [y_i - \bar y]^2.
$$

* $y_i$ is the $i^{\rm th}$ observed $y$ value (data point)
* $x_i$ is the $i^{\rm th}$ observed $x$ value (data point)
* $y$ is the best-fit function
* $\bar y$ is the mean of the observed $y$ values

In [ ]:
def R2(f, ydata, xdata):
    SSres = np.sum((ydata - f(xdata)) ** 2)
    SStot = np.sum((ydata - np.mean(ydata)) ** 2)
    return 1.0 - SSres / SStot

In [ ]:
r2_1, r2_2, r2_3 = [R2(f, ydata, xdata) for f in [y1, y2, y3]]

plt.plot(xdata, ydata, "bx")
plt.plot(x, y1(x), label=f"$R^2 = {r2_1:.4g}$")
plt.plot(x, y2(x), label=f"$R^2 = {r2_2:.4g}$")
plt.plot(x, y3(x), label=f"$R^2 = {r2_3:.4g}$")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()

### Beware of over-fitting

* One way is to instead optimse the _modified_ least squares

$$
  SS_{{\rm res},M} = \sum_i \frac{[y_i - y(x_i)]^2}{N - 2M}
$$

  * $N$ is the number of data points
  * $M$ is the number of parameters (e.g., $m=2$ for linear, 3 for quadratic, 4 for cubic)

* General rule: use as _few_ parameters as you can, never more

In [ ]:
coefs6 = np.polyfit(xdata, ydata, 6)
y6 = np.poly1d(coefs6)
r2_6 = R2(y6, ydata, xdata)

SSm1 = np.sum((ydata - y1(xdata)) ** 2) / (len(xdata) - 2 * 2)
SSm2 = np.sum((ydata - y2(xdata)) ** 2) / (len(xdata) - 2 * 3)
SSm3 = np.sum((ydata - y3(xdata)) ** 2) / (len(xdata) - 2 * 4)
SSm8 = np.sum((ydata - y6(xdata)) ** 2) / (len(xdata) - 2 * 6)

plt.plot(xdata, ydata, "bx")
plt.plot(x, y1(x), label=f"$R^2 = {r2_1:.4g}, SS_m = {SSm1:.2g}$")
plt.plot(x, y2(x), label=f"$R^2 = {r2_2:.4g}, SS_m = {SSm2:.2g}$")
plt.plot(x, y3(x), label=f"$R^2 = {r2_3:.4g}, SS_m = {SSm3:.2g}$")
plt.plot(x, y6(x), label=f"$R^2 = {r2_6:.4g}, SS_m = {SSm8:.2g}$")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()

### Beware extrapolting

* Polynomials are very unstable for large arguments

In [ ]:
x2 = np.linspace(0, 200, 100)

plt.plot(xdata, ydata, "bx")
plt.plot(x2, y1(x2), label=f"linear")
plt.plot(x2, y2(x2), label=f"quadratic")
plt.plot(x2, y3(x2), label=f"cubic")
plt.plot(x2, y6(x2), label=f"order 6")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.xlim(0, 200)
plt.ylim(0, 40)
plt.legend()
plt.show()

----------------------------------------------
## Significance, T-test, etc.

If the standard deviation of the population is not well known, it's common to use a student t distribution instead of a Gaussian:
* https://en.wikipedia.org/wiki/Student%27s_t-distribution

If in the null hyposesis, the mean of our observations would be expected to be 0,
we form the t-statistic:
$$
  t = \frac{\bar x}{\sigma_{\bar x}}
$$

where 
* $\bar x$ is the observed mean
* $\sigma_{\bar x} = \sigma_{x} / \sqrt{N}$ is the _standard error_ in the mean
* $\sigma_{x}$ is the (unbiased/corrected) sample standard deviation of observed $x$
* $N$ is the number of observations

We wish to know:

 * For a given $t$ what is the significance (i.e., what is the $p$ value)
 * For a given $p$ value/significance, what $t$ score is required

This depends on the degrees of freedom
 * d.o.f = number of data points, minus number of fit parameters
 * As d.o.f gets large, approaches Gaussian

See:

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

* https://en.wikipedia.org/wiki/Student%27s_t-distribution#Table_of_selected_values

In [ ]:
# see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html


# Given a target p value, finds the required t score:
def find_t(p_target, dof):
    """Returns the t value required to achieve p value of p_target,
    given dof degrees of freedom (dof = N_data - N_params)"""
    return np.abs(stats.t.ppf(0.5 * p_target, dof))


# Given a t-score, finds the p-value
def find_p(t, dof):
    """Returns the p value, given the t value"""
    return 2 * (1 - np.abs(stats.t.cdf(t, dof)))


# Compare to: https://en.wikipedia.org/wiki/Student%27s_t-distribution#Table_of_selected_values
print("Table of t values for the 85%, 95%, and 99% confidence levels")
print("  dof   t(80%)  t(95%)  t(95%)")
for dof in [10, 20, 50, 100, 120, 10000]:
    print(
        f"{dof:5d}   {find_t(0.2, dof):.3f}   {find_t(0.05, dof):.3f}   {find_t(0.01, dof):.3f}"
    )

# Example
print()
print("For example, our cat data in linear model:")
n_data = len(xdata)
num_params = 3
degrees_of_freedom = n_data - num_params

tcat = find_t(0.05, degrees_of_freedom)
print(f"cat data: We require t={tcat:.2f} for a 95% C.L.")

----------------
## Linear regression

* In the simples (linear) case 'linregress' method does it all for us
* Calculates best-fit slope + intercept, with uncertainties, and p-values etc:
* nb: p value alone not very meaningful here

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

In [ ]:
linreg = stats.linregress(xdata, ydata)
print("R=", linreg.rvalue, "p=", linreg.pvalue)
print("b=", linreg.intercept, "b_err=", linreg.intercept_stderr)
print("m=", linreg.slope, "m_err=", linreg.stderr)

t = linreg.rvalue * np.sqrt((len(xdata) - 2) / (1 - linreg.rvalue**2))


print("Manual t score: ", t)
print("Manual p value: ", find_p(t, len(xdata) - 2))

best = linreg.intercept + linreg.slope * x


# Error in line y = a + b*x, given error in a and b:
def err(x, da, db):
    return np.sqrt(da**2 + (db * x) ** 2)


# The (standard) uncertainty in y
dy1 = err(x, linreg.intercept_stderr, linreg.stderr)

# t value at 95% and 99% confidence level:
# Note: these differ only slightly from Gaussian values.
# It's fine to just use the 68/95/99.7 rule
t_95 = find_t(0.05, len(xdata) - 2)
t_99 = find_t(0.01, len(xdata) - 2)

plt.plot(xdata, ydata, "bx")
plt.plot(x, best, "r-", label="best fit")
plt.plot(x, best + dy1, "k-", label="Standard error")
plt.plot(x, best - dy1, "k-")
plt.plot(x, best + t_95 * dy1, "--", c="grey", label="95% confidence")
plt.plot(x, best - t_95 * dy1, "--", c="grey")
plt.plot(x, best + t_99 * dy1, ls="dotted", c="lightgrey", label="99% confidence")
plt.plot(x, best - t_99 * dy1, ls="dotted", c="lightgrey")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()

### More general method: use curve_fit

* Fits _any_ function (not just a polynomaial) to data using least squares
* optionally include weights for the fit

Returns the set of fit parameters `popt`, and covariance matrix `pcov`
* The diagonal entries in the covariance matrix are approximately the variances in the best-fit parameters
* e.g., for parameters a, b, c, ... get their (standard, 1sigma) uncertainties:
* `da, db, dc = np.sqrt(np.diag(pcov))`

See:
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
def f_linear(x, a, b):
    return a + b * x


# perform the fit:
popt, pcov = curve_fit(f_linear, xdata, ydata)

# extract the parameters from 'popt = optimised paramters' (careful of the order)
a, b = popt

# Extract the _approximate_ (1 sigma = standard) uncertainties from pcov (parameter covariance)
da, db = np.sqrt(np.diag(pcov))


def df(x, da, db):
    return np.sqrt(da**2 + (db * x) ** 2)


# t scores required for 95, 99 % confidence levels
# Note: these differ only slightly from Gaussian values
t_95 = find_t(0.05, len(xdata) - 2)
t_99 = find_t(0.01, len(xdata) - 2)

# Best fit and error in fit:
best = f_linear(x, a, b)
error = df(x, da, db)


print("Roughly: t = ", b / db)

plt.title(f"Linear Fit: $y = a + bx$")
plt.text(10, 12, f"$a={a:.3f}\\pm{da:.3f}$\n$b={b:.3f}\\pm{db:.3f}$")

plt.plot(xdata, ydata, "bx")
plt.plot(x, best, "r-", label="best fit")

# Plot standard errors:
plt.plot(x, best + error, "k-", label="Standard error")
plt.plot(x, best - error, "k-")

# Confidence intervals
plt.plot(x, best + t_95 * error, c="grey", ls="dashed", label="95% C.L.")
plt.plot(x, best - t_95 * error, c="grey", ls="dashed")
plt.plot(x, best + t_99 * error, ls="dotted", c="lightgrey", label="99% confidence")
plt.plot(x, best - t_99 * error, ls="dotted", c="lightgrey")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()

#### ... and for a quadratic fit

* Note: uncertainty in Y(x) for the fit becomes less meaningful
* Calculation assumes uncertainty in coeficients is independent
    * this is certainly not true!
* Proper statistics is hard, this is OK as a start

In [ ]:
def f_quad(x, a, b, c):
    return a + b * x + c * x**2


# perform the fit:
popt, pcov = curve_fit(f_quad, xdata, ydata)

# extract the parameters from 'popt = optimised paramters' (careful of the order)
a, b, c = popt

# Extract the _approximate_ (1 sigma = standard) uncertainties from pcov (parameter covariance)
da, db, dc = np.sqrt(np.diag(pcov))


# Uncertainty in f(x), given uncertainty in paramaters [just add in quadrature]
# nb: assumes independent, not really true!
def df(x, da, db, dc):
    return np.sqrt(da**2 + (db * x) ** 2 + (dc * x**2) ** 2)


best = f_quad(x, a, b, c)
error = df(x, da, db, dc)

# t values, now with 3 parameters
t_95 = find_t(0.05, len(xdata) - 3)
t_99 = find_t(0.01, len(xdata) - 3)

plt.title("Fit: $y = a + bx + cx^2$")
plt.plot(xdata, ydata, "bx")
plt.plot(x, best, "r-", label="best fit")
plt.text(
    10, 15, f"$a={a:.2f}\\pm{da:.2f}$\n$b={b:.3f}\\pm{db:.3f}$\n$c={c:.2g}\\pm{dc:.2g}$"
)
# 1 sigma (standard) errors:
plt.plot(x, best + error, "k-", label="Standard error")
plt.plot(x, best - error, "k-")
# Confidence intervals
plt.plot(x, best + t_95 * error, c="grey", ls="dashed", label="95% C.L.")
plt.plot(x, best - t_95 * error, c="grey", ls="dashed")
plt.plot(x, best + t_99 * error, ls="dotted", c="lightgrey", label="99% confidence")
plt.plot(x, best - t_99 * error, ls="dotted", c="lightgrey")
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()

### Plot with fill_between

In [ ]:
plt.title("Fit: $y = a + bx + cx^2$")
plt.plot(xdata, ydata, "bx")
plt.plot(x, best, "r-", label="best fit")
plt.text(
    10, 15, f"$a={a:.2f}\\pm{da:.2f}$\n$b={b:.3f}\\pm{db:.3f}$\n$c={c:.2g}\\pm{dc:.2g}$"
)
# 1 sigma (standard) errors:
plt.plot(x, best + error, ls="dotted", c="lightgrey")
plt.plot(x, best - error, ls="dotted", c="lightgrey")
plt.plot(x, best + t_95 * error, ls="dotted", c="lightgrey")
plt.plot(x, best - t_95 * error, ls="dotted", c="lightgrey")
plt.plot(x, best + t_99 * error, ls="dotted", c="lightgrey")
plt.plot(x, best - t_99 * error, ls="dotted", c="lightgrey")
plt.fill_between(x, best + error, best - error, color="red", alpha=0.4, label="Error")
plt.fill_between(
    x, best + t_95 * error, best - t_95 * error, color="red", alpha=0.25, label="95%"
)
plt.fill_between(
    x, best + t_99 * error, best - t_99 * error, color="red", alpha=0.1, label="99%"
)
plt.xlabel("Retina area (mm$^2$)")
plt.ylabel("CP ratio")
plt.legend()
plt.show()